In [1]:
#load data
import numpy as np
celeba = np.load('drive/My Drive/celeba.npy')

In [2]:
#last 10k images as test set
celeba_test = celeba[192599:202599]

In [3]:
#rescale as 0 1
celeba_test = (celeba_test-127.5)/127.5

In [10]:
from skimage.transform import resize
import copy
from numpy.random import randint
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy.random import randn
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.models import load_model




In [12]:
#RUN FOR GAN
# generate points in latent space as input for the generator
def generate_latent_points(latent_dimension, n_batch):
  
  # generate points in the latent space
  lat_input = randn(latent_dimension * n_batch)
  
  # reshape into a batch of inputs for the network
  lat_input = lat_input.reshape(n_batch, latent_dimension)
  
  return lat_input

In [5]:
#RUN FOR CGAN
attribute = np.loadtxt('drive/My Drive/list_attr_celeba.txt',skiprows=2,unpack=True,usecols=range(1,41))
attribute = np.transpose(attribute)
attribute[attribute==-1]=0


# generate points in latent space as input for the generator
def generate_latent_points(latent_dim,labels_dataset, n_samples, n_labels = 40):
  
  # generate points in the latent space
  x_input = randn(latent_dim * n_samples)
  
  # reshape into a batch of inputs for the network
  x_input = x_input.reshape(n_samples, latent_dim)

  #select random labels from the dataset
  idx = randint(0, labels_dataset.shape[0], n_samples)
  labels = labels_dataset[idx]
  
  
  return [x_input,labels]


In [ ]:
model = load_model('drive/My Drive/model/ACgenerator_model_040.h5')

In [7]:
# compute frechet inception distance
def compute_fid(act1, act2):
  
  # compute mean and covariance statistics
 
   
  mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
  mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
  
  # calculate sum squared difference between means
  ssdiff = np.sum((mu1 - mu2)**2.0)
  
  # calculate sqrt of product between cov
  covmean = sqrtm(sigma1.dot(sigma2))
  
  # check and correct imaginary numbers from sqrt
  if iscomplexobj(covmean):
    covmean = covmean.real
  
  # calculate score
  fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
  return fid

In [ ]:
#compute activations of test set
model2 = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))
actreal = []
for i in range(1000):
  print(i)
  temp = copy.deepcopy(celeba_test[i*10:((i*10)+10)])
  
  temp1 = np.array([resize(err,(299,299,3),1) for err in temp])
  
  temp1 = model2.predict(temp1)
  
  actreal.append(temp1)


In [ ]:
#RUN FOR GAN
latent_dim = 100
latent_points = generate_latent_points(latent_dim, 10000)

# generate images
fake = model.predict(latent_points)




 

In [11]:
#RUN FOR CGAN
latent_dim = 100
[latent_points,label_temp] = generate_latent_points(latent_dim,attribute,n_samples = 10000)

# generate images
fake = model.predict([latent_points,label_temp])



In [ ]:
#compute activations of generated images
actfake = []
for i in range(1000):
  print(i)
  temp = copy.deepcopy(fake[i*10:((i*10)+10)])
  
  temp1 = np.array([resize(err,(299,299,3),1) for err in temp])
  
  temp1 = model2.predict(temp1)
  
  actfake.append(temp1)


In [13]:
#compute FID
actreal = np.array(actreal)
actfake = np.array(actfake)

actreal = actreal.reshape((10000,2048))
actfake = actfake.reshape((10000,2048))

# fid between act1 and act2
fid = compute_fid(actreal, actfake)
print('FID: %.3f' % fid)

FID: 12.071
